# Generate Data for Chartbook

Brian Dew

@bd_econ

In [1]:
import sys
sys.path.append('../src')

import uschartbook.config

from uschartbook.config import *
from uschartbook.utils import *

In [2]:
#nipa_series_codes(retrieve_table('T11500'))

### GDPNow

In [3]:
url = ('https://api.stlouisfed.org/fred/series?series_id=GDPNOW&'+
       f'api_key={fred_key}&file_type=json')
r = requests.get(url).json()
update = r['seriess'][0]['last_updated']
url = ('https://api.stlouisfed.org/fred/series/observations?series_id=GDPNOW&'+
       f'api_key={fred_key}&file_type=json')
r = requests.get(url).json()
lv = pd.DataFrame(r['observations']).set_index('date')['value'].sort_index()[-1]
lvt = f'{float(lv):.1f}'
nowdt = pd.DataFrame(r['observations']).set_index('date')['value'].sort_index().index[-1]
ltdt = dtxt(nowdt)['qtr1']

# Store GDP now value
gdpnow = pd.Series({'date': nowdt, 'value': lv}).to_frame().T.set_index('date')
gdpnow.to_csv(data_dir / 'gdpnow.csv', index_label='date')

s = ['A191RL']
gdp = nipa_df(retrieve_table('T10502')['Data'], s).sort_index()
gdpdt = gdp.index[-1]

In [4]:
# Only add mark to plot if nowcast is available
mark = ' (see \\tikz \draw[black, fill=red] (2.5pt,2.5pt) circle (2.5pt);)'
if dtxt(gdpdt)['datetime'] < nowdt:
    linedt = dtxt(pd.to_datetime(nowdt) - pd.DateOffset(days=46))['datetime']
    node = (f'\\node[label={{[align=left]90:{{\scriptsize \\textit{{{lvt}}}}}}}, '+
            'circle, draw=black, fill=red, inner sep=1.8pt] at '+
            f'(axis cs:{nowdt}, {lv}) {{}};'+
            f'\draw [dashed] (axis cs:{{{linedt}}},\pgfkeysvalueof{{/pgfplots/ymin}})'+
            f' -- (axis cs:{{{linedt}}}, \pgfkeysvalueof{{/pgfplots/ymax}});')
    write_txt(text_dir / 'gdpnow_node.txt', node)

    node = ('\\node[circle, draw=black, fill=red, inner sep=1.0pt] '+
            f'at (axis cs:{nowdt}, {lv}) {{}};')
    write_txt(text_dir / 'gdpnow_node2.txt', node)
    # Nowcast text
    write_txt(text_dir / 'nowcast_text.txt', 'nowcast')
    latest_txt = 'latest '

else:
    node = ''
    write_txt(text_dir / 'gdpnow_node.txt', node)
    write_txt(text_dir / 'gdpnow_node2.txt', node)
    write_txt(text_dir / 'nowcast_text.txt', '\color{white}{.}')
    latest_txt = ''
    mark = ''
            
text = (f'The {latest_txt}nowcast for {ltdt} is {lvt} percent, as '+
        f'of {dtxt(pd.to_datetime(update))["day1"]}{mark}.')
write_txt(text_dir / 'gdpnow.txt', text)
print(text)

The latest nowcast for 2023 Q4 is 2.4 percent, as of January 18, 2024 (see \tikz \draw[black, fill=red] (2.5pt,2.5pt) circle (2.5pt);).


### Monthly nominal GDP measure

Does not show meaningful differences within months, basic interpolation. Adds latest quarter from nowcasts.

In [5]:
pce = pd.read_csv(data_dir / 'pce_now.csv', index_col='date', parse_dates=True)
gdp = gdpnow.copy().astype('float')
gdp.index = pd.to_datetime(gdp.index)
data = pd.concat([gdp, pce], axis=1)
data['real'] = data[['value', 'pce']].sum(axis=1)

In [6]:
df = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])['A191RC']
mult = ((data['real'].iloc[0] / 4) / 100) + 1
df.loc[data.index[0]] = df.iloc[-1] * mult
df.index = (df.index + pd.DateOffset(months=2)).to_period('M')
mon = df.resample('M').interpolate()
mon.index = mon.index.to_timestamp()
mon.to_csv(data_dir / 'gdp_monthly.csv', index_label='date')

### Estimate of labor productivity

In [7]:
data = pd.read_csv(data_dir / 'gdpjobslvl.csv', 
                   index_col='date', parse_dates=True)
currdt = data['GDP'].dropna().index[-1]
gdpnow = pd.read_csv(data_dir / 'gdpnow.csv', index_col='date', parse_dates=True)

qch = ((gdpnow['value'].iloc[0] / 100) + 1) ** (1/4)
nowdt = gdpnow.index[0]
ltmo = f'{gdpnow.index[0].year}-{gdpnow.index[0].quarter*3}-01'
prdt = nowdt - pd.DateOffset(months=3)

# Fill in data based on GDPNow estimate
if pd.isna(data.loc[nowdt, 'GDP']):
    if pd.isna(data.loc[nowdt, 'TOT_HRS']):
        popch = (data['POP'].pct_change().iloc[-1]) + 1
        data.loc[nowdt, 'TOT_HRS'] = data.loc[prdt, 'TOT_HRS'] * popch
        data.loc[nowdt, 'EPOP_sa'] = data.loc[prdt, 'EPOP_sa']
        data.loc[nowdt, 'AAH_trend'] = data.loc[prdt, 'AAH_trend']
    data.loc[nowdt, 'GDP'] = data.loc[prdt, 'GDP'] * qch
    data.loc[nowdt, 'LPROD'] = (data.loc[nowdt, 'GDP'] / 
                                data.loc[nowdt, 'TOT_HRS'])
    qtrs = -10
    gdp_now = True
else:
    qtrs = -9
    gdp_now = False
    
# Contribution to GDP growth
srs = {'EPOP_sa': 'epop_contr', 'POP': 'pop_contr', 
       'AAH_trend': 'hours_contr', 'lprod': 'prod', 
       'GDP': 'gdp'}
dft = data.copy()[['GDP', 'POP', 'EPOP_sa', 'AAH_trend', 
                   'TOT_HRS', 'LPROD']].dropna()
c = ((((dft.pct_change() + 1) ** 4) - 1) * 100).rename(srs, axis=1).dropna()
clt = c.copy().iloc[qtrs:]

# Nodes
sl = ['pop_contr', 'epop_contr', 'hours_contr', 'LPROD']
nodes = selected_nodes(clt[sl], nowrow=gdp_now, threshold=0.3)
write_txt(text_dir / 'gdp_hh_comp_recent_nodes.txt', nodes)

# Save csv
clt['label'] = [f'Q{i.quarter} \\\\ {i.year}' 
                  if (i.quarter == 1) | (i == clt.index[0]) 
                  else dtxt(i)['qtr3'] for i in clt.index]
clt.to_csv(data_dir / 'gdp_hh_comp_recent.csv', index_label='date')

# Convert to indices
for i in ['LPROD','TOT_HRS', 'GDP', 
          'POP', 'EPOP_sa', 'AAH_trend']:
    data[f'{i}_ix'] = (data[i] / data.loc['2000-01-01', i]) * 100

# Calculate productivity growth rate
data['LPROD_gr'] = c.LPROD
    
# Create table
cl = c_line('cyan!70!white', see=False, paren=False)
names = {'LPROD_ix': f'\hspace{{0.1mm}}{cl} Labor Productivity (index)', 
         'GDP_ix': '\hspace{4mm} Real GDP (index)',
         'TOT_HRS_ix': '\hspace{4mm} Total Hours Worked (index)', 
         'POP': '\hspace{7mm} Population (millions)', 
         'EPOP_sa': '\hspace{7mm} Employment Rate (percent)', 
         'AAH_trend': '\hspace{7mm} Average Workweek (hours)',
         'LPROD_gr': '\hspace{0.1mm} Labor Productivity Growth (percent)'}

table = data[names.keys()].dropna().iloc[[-1,-2,-3,-5]].T
table['2019-10-01'] = data.loc['2019-10-01', names.keys()]
table.columns = [dtxt(c)['qtr1'] for c in table.columns]
table['2014'] = data.loc['2014', names.keys()].mean()
table['1989'] = data.loc['1989', names.keys()].mean()
table = table.rename(names)
nowqt = dtxt(nowdt)['qtr1']
if nowdt > currdt:
    table[nowqt] = (table[nowqt].apply('\\textit{{{:.1f}}}'.format))
    table.iloc[:, 1:] = table.iloc[:, 1:].applymap('{:.1f}'.format)
    table.columns = table.columns.str.replace(nowqt, f'\\\\textit{{{{{nowqt}}} Est.}}')
else:
    table = table.applymap('{:.1f}'.format)
table.to_csv(data_dir / 'gdpjobs.tex', sep='&', 
             lineterminator='\\\ ', quotechar=' ')

In [8]:
# Only add mark to plot if nowcast is available
mark = ' (see \\tikz \draw[black, fill=cyan!70!white] (2.5pt,2.5pt) circle (2.5pt);)'
prval = data.loc[currdt, 'LPROD_gr']
nowval = data.loc[nowdt, 'LPROD_gr']
lv = data.loc[nowdt, 'LPROD_ix']
lvt = f'{lv:.1f}'
cht = f'{nowval:.1f}'
if currdt < nowdt:
    node = (f'\\node[label={{[align=left]90:{{\scriptsize \\textit{{{lvt}}}}}}}, '+
            'circle, draw=black, fill=cyan!70!white, inner sep=1.2pt] at '+
            f'(axis cs:{nowdt.date()}, {lv}) {{}};')
    node2 = (f'\\node[label={{[align=left]90:{{\scriptsize \\textit{{{cht}}}}}}}, '+
            'circle, draw=black, fill=cyan!70!white, inner sep=1.2pt] at '+
            f'(axis cs:{nowdt.date()}, {nowval}) {{}};')
    txt = (f'The estimate for {nowqt}{mark}, based on the Federal Reserve Bank of '+
        'Atlanta GDPNow, suggests annualized productivity growth of '+
        f'{nowval:.1f} percent.')
    write_txt(text_dir / 'gdpjobs_est_node.txt', node)
    write_txt(text_dir / 'lprod_rec_node.txt', node2)
else:
    node = ''
    txt = ''
    write_txt(text_dir / 'gdpjobs_est_node.txt', node)

text = ('More-recent data show annualized total economy productivity '+
        f'growth of {prval:.1f} percent in {dtxt(currdt)["qtr1"]}. {txt}')
write_txt(text_dir / 'gdpjobs_est.txt', text)
print(text)

More-recent data show annualized total economy productivity growth of 4.9 percent in 2023 Q3. The estimate for 2023 Q4 (see \tikz \draw[black, fill=cyan!70!white] (2.5pt,2.5pt) circle (2.5pt);), based on the Federal Reserve Bank of Atlanta GDPNow, suggests annualized productivity growth of 2.5 percent.


In [9]:
rec = data.loc[:currdt, 'LPROD_gr'].iloc[-8:].to_frame()
rec['Quarter'] = [f'Q{i.quarter}\\\\{i.year}' if i.quarter == 1 
                  else dtxt(i)['qtr3'] for i in rec.index]
# From GDP Now (cover cases where no nowcast available)
if currdt < nowdt:
    next_qtr = rec.index[-1] + pd.DateOffset(months=3)
    label = (f'Q{next_qtr.quarter}\\\\{next_qtr.year}' 
             if next_qtr.quarter == 1 else dtxt(next_qtr)['qtr3'])
    frow = pd.DataFrame({'Quarter': label, 
                         'date': next_qtr}, index={'date': 4})
    rec= pd.concat([rec.reset_index(), frow]).set_index('date')
    rec.loc[next_qtr, 'LPROD_gr'] = 0
rec['zero'] = 0
rec.to_csv(data_dir / 'lprod_rec.csv', index_label='date')

# Average bar
start_date = rec.index[0] - pd.DateOffset(months=2)
end_date = rec.index[-1] + pd.DateOffset(months=1)
val = data['LPROD_gr'].iloc[:-1].astype('float').mean()
color = 'gray!60!white'
bar = (f'\draw [{color}] (axis cs:{{{start_date}}},{val}) -- '+
       f'(axis cs:{{{end_date}}},{val});')
bardf = pd.Series(index=[start_date, end_date], 
                data=[val, val], name='Bar')
node = end_node(bardf, color, loc='start')
write_txt(text_dir / 'lprod_rec_bar_node.txt', bar + '\n' + node)

In [10]:
# Optional GDPNow text for HH inp contributions
text = ''
if currdt < nowdt:
    sl = ['pop_contr', 'epop_contr', 'hours_contr', 'LPROD']
    ltvals = clt[sl].iloc[-1]

    epopnm = ('the higher employment rate' if ltvals.epop_contr > 0.4 
              else 'the lower employment rate' if ltvals.epop_contr < -0.4
              else 'the employment rate')
    hoursnm = ('longer average workweeks' if ltvals.hours_contr > 0.4 
              else 'the drop in hours worked' if ltvals.hours_contr < -0.4
              else 'hours worked')
    rn = {'epop_contr': epopnm,
          'pop_contr': 'population growth',
          'hours_contr': hoursnm,
          'LPROD': 'labor productivity'}
    ltvals = ltvals.rename(rn)

    sel = ltvals[(abs(ltvals) > 0.4).sort_values()]
    if len(sel) > 0:
        cat1 = sel.index[-1]
        cv1 = value_text(sel.sort_values().iloc[-1], 'plain', ptype='pp')
        ext = '.'
    if len(sel) > 1:
        cat2 = sel.index[-2]
        cv2 = value_text(sel.sort_values().iloc[-2], 'plain', ptype='pp')
        ext = f' and {cv2} from {cat2}.'
    nowdtt = dtxt(nowdt)['qtr1']
    gdpch = (value_text(clt.gdp.iloc[-1], 'increase_by', threshold=0.1)
             .replace('ed by', 'e by').replace('was', 'be'))
    text = ('Using the Atlanta Fed GDPNow and the latest available population '+
            'and labor force data, we can estimate contributions to growth for '+
            f'{nowdtt}. Real GDP is estimated to {gdpch}, with '+
            f'contributions of {cv1} from {cat1}{ext}')
write_txt(text_dir / 'gdpjobs_ch_est.txt', text)
print(text)

Using the Atlanta Fed GDPNow and the latest available population and labor force data, we can estimate contributions to growth for 2023 Q4. Real GDP is estimated to increase by 2.4 percent, with contributions of 2.5 percentage points from labor productivity and 0.6 percentage point from population growth.


### GDP growth rate

In [11]:
s = ['A191RL']
df = nipa_df(retrieve_table('T10502')['Data'], s).sort_index()
df.loc['1989':].to_csv(data_dir / 'gdp.csv', index_label='date')
date = dtxt(df.index[-1])['qtr1']

txt = f'{date}: {df["A191RL"].iloc[-1]}\%'
write_txt(data_dir / 'gdp.txt', txt)

In [12]:
dfa = (nipa_df(retrieve_table('T10101A')['Data'], ['A191RL'])
       ['A191RL'])
dfa.index = dfa.index + pd.DateOffset(months=6)
dfa.loc['1989':].to_csv(data_dir / 'gdp_a.csv', index_label='date')

In [13]:
df = nipa_df(retrieve_table('T10502')['Data'],
             ['A191RL'])['A191RL']
dfl = (nipa_df(retrieve_table('T10106')['Data'], ['A191RX'])
       .loc['1989':, 'A191RX'])
# Dates for latest quarters chart
a = df.iloc[-4:].to_frame()
next_qtr = a.index[-1] + pd.DateOffset(months=3)
frow = pd.DataFrame({'index': next_qtr}, index={'index': 4})
# From GDP Now (cover cases where no nowcast available)
if gdpdt < nowdt:
    a = pd.concat([a.reset_index(), frow]).set_index('index')
    a.loc[next_qtr, 'A191RL'] = 0
a['zero'] = 0
a['Quarter'] = [f'Q{i.quarter}\\\\{i.year}' 
                if (i.quarter == 1) | (i == a.index[0])
                else dtxt(i)['qtr3'] for i in a.index]
a.to_csv(data_dir/'gdp_rec.csv', index_label='date')
# Text
lty = df.index[-1].year
ltdt = dtxt(df.index[-1])['qtr2']
ltdt2 = dtxt(df.index[-1])['qtr1']
gtot = cagr(dfl)
g9300 = cagr(dfl.loc['1993':'2000'])
g0013 = cagr(dfl.loc['2001':'2013'])
g1419 = cagr(dfl.loc['2014':'2019'])
# expenditure approach data
d19 = dfl.loc['2019-10-01':]
g20on = cagr(d19)
cov1 = value_text(df.loc['2020-04-01'])
cov2 = value_text(df.loc['2020-07-01'], 'increase_by')
ltval = value_text(df.iloc[-1], adj='annual')
prval = prval_comp(df)

color='red'
url = 'https://www.bea.gov/data/gdp/gross-domestic-product'
text = (f'Real GDP growth \href{{{url}}}{{measures}} changes in '+
        'economic activity. As seen in the previous subsection, '+
        'real GDP has increased steadily over the long-term. '+
        'Since 1989, growth averages '+
        f'{gtot:.1f} percent per year {c_box(color)}. '+
        'Growth rates were relatively high during the mid- '+
        f'to late-1990s, averaging {g9300:.1f} percent '+
        'from 1993 to 2000.\n\nIn the 2000s, the housing '+
        'bubble boosted GDP but then collapsed, '+
        f'leading to average growth of only {g0013:.1f} '+
        'percent from 2001 to 2013. Growth was slightly '+
        f'stronger from 2014 to 2019, averaging {g1419:.1f} '+
        'percent per year.\n\nIn 2020, COVID-19 caused '+
        'an economic shutdown, followed by monetary '+
        'and fiscal stimulus, resulting in large swings in '+
        f'GDP. Annualized real GDP {cov1} in Q2, and {cov2} in Q3, '+
        'by far the largest changes in recent history. Since 2019 Q4, '+
        f'real GDP has grown at an average annual rate of {g20on:.1f} '+
        'percent.')
write_txt(text_dir / 'gdp_gr.txt', text)
print(text)

txt2 = ('The bottom-left chart shows annual growth, to make '+
        'trends more visible. The bottom-right chart shows '+
        'the most-recent four quarters and the estimate for the '+
        'current quarter. In the \\textbf{latest data}, '+
        f'covering {ltdt}, real GDP {ltval}, compared to {prval}.')
write_txt(text_dir / 'gdp_gr2.txt', txt2)
print('\n', txt2)

Real GDP growth \href{https://www.bea.gov/data/gdp/gross-domestic-product}{measures} changes in economic activity. As seen in the previous subsection, real GDP has increased steadily over the long-term. Since 1989, growth averages 2.4 percent per year (see\cbox{red}). Growth rates were relatively high during the mid- to late-1990s, averaging 3.9 percent from 1993 to 2000.

In the 2000s, the housing bubble boosted GDP but then collapsed, leading to average growth of only 1.9 percent from 2001 to 2013. Growth was slightly stronger from 2014 to 2019, averaging 2.7 percent per year.

In 2020, COVID-19 caused an economic shutdown, followed by monetary and fiscal stimulus, resulting in large swings in GDP. Annualized real GDP decreased 28.0 percent in Q2, and increased by 34.8 percent in Q3, by far the largest changes in recent history. Since 2019 Q4, real GDP has grown at an average annual rate of 1.9 percent.

 The bottom-left chart shows annual growth, to make trends more visible. The bot

In [14]:
# Dates for latest quarters chart
a = df.iloc[-8:].to_frame()
a['Quarter'] = [f'Q{i.quarter}\\\\{i.year}' if i.quarter == 1 
                  else dtxt(i)['qtr3'] for i in a.index]
# From GDP Now (cover cases where no nowcast available)
if gdpdt < nowdt:
    next_qtr = a.index[-1] + pd.DateOffset(months=3)
    label = (f'Q{next_qtr.quarter}\\\\{next_qtr.year}' 
             if next_qtr.quarter == 1 else dtxt(next_qtr)['qtr3'])
    frow = pd.DataFrame({'Quarter': label, 
                         'index': next_qtr}, index={'index': 4})
    a = pd.concat([a.reset_index(), frow]).set_index('index')
    a.loc[next_qtr, 'A191RL'] = 0
a['zero'] = 0
a.to_csv(data_dir/'gdp_rec2.csv', index_label='date')

# Average bar
start_date = a.index[0] - pd.DateOffset(months=2)
end_date = a.index[-1] + pd.DateOffset(months=1)
val = df.mean()
color = 'gray!60!white'
bar = (f'\draw [{color}] (axis cs:{{{start_date}}},{val}) -- '+
       f'(axis cs:{{{end_date}}},{val});')
bardf = pd.Series(index=[start_date, end_date], 
                data=[val, val], name='Bar')
node = end_node(bardf, color, loc='start')
write_txt(text_dir / 'gdp_rec2_bar_node.txt', bar + '\n' + node)

### Atlanta Fed GDPNow Contributions

In [15]:
# GDP Now contributions to GDP
url = ('https://www.atlantafed.org/-/media/documents/cqer/researchcq/'+
       'gdpnow/GDPTrackingModelDataAndForecasts.xlsx')
df = pd.read_excel(url, sheet_name='ContribHistory', header=0, skipfooter=9)
title = df.iloc[0, 0]
df = df.iloc[1:]
d = {'Personal consumption expenditures (PCE)': 'DPCERY',
     'Gross Private Domestic Investment (GPDI)': 'A006RY',
     'Government expenditures': 'A822RY',
     'Change in net exports': 'A019RY',
     'GDP Nowcast': 'A191RL'}
res = df.dropna(subset=df.columns[2])
names = res[res.columns[1]].to_list()
res.index = [n.split('-', 1)[-1].strip().rstrip('*') for n in names]
res = res.iloc[:, 2:].T
res.index = pd.to_datetime(res.index)
nowcomp = res[d.keys()].iloc[-1].rename(d)
nowcomp.name = pd.to_datetime(title[-6:])
update = nowcomp.name
nowcomp = nowcomp.to_frame().T

# Text based on GDPNow
nowval = nowcomp.iloc[-1]
nowdt = dtxt(nowcomp.index[-1])['qtr1']
gdpch = value_text(nowval['A191RL'], 'plain')
csch = value_text(nowval['DPCERY'], 'contribution_of', 
                  ptype='pp', digits=2, threshold=0.1)
gdpich = value_text(nowval['A006RY'], 'contribution_of', 
                  ptype='pp', digits=2, threshold=0.1)
govch = value_text(nowval['A822RY'], 'contribution_of', 
                  ptype='point', digits=2, threshold=0.1)
nxch = value_text(nowval['A019RY'], 'contribution_of', 
                  ptype='point', digits=2, threshold=0.1)
text = ('The Federal Reserve Bank of Atlanta GDPNow estimate for '+
        f'{nowdt} of {gdpch} is based on {csch} '+
        f'from consumer spending, {gdpich} '+
        f'from private investment, {govch} '+
        f'from government, and {nxch} from net exports. ')
write_txt(text_dir / 'gdp_comp_exp_now.txt', text)
print(text)

The Federal Reserve Bank of Atlanta GDPNow estimate for 2023 Q4 of 2.4 percent is based on a contribution of 1.92 percentage points from consumer spending, a subtraction of 0.12 percentage point from private investment, a contribution of 0.68 point from government, and virtually no contribution from net exports. 


In [16]:
# Contribution to growth from expenditure approach categories
s = ['DPCERY', 'A006RY', 'A822RY', 'A019RY', 'A191RL']
df = nipa_df(retrieve_table('T10502')['Data'], s)
df.loc['1989':].to_csv(data_dir / 'comp.csv', index_label='date')
# Annual GDP version 
dfa = nipa_df(retrieve_table('T10502A')['Data'], s).loc['1989':]
dfa.to_csv(data_dir / 'gdp_comp_ann.csv', index_label='date')
# Recent data
dfr = df.copy().iloc[-9:]
if (dfr.index[-1] < nowcomp.index[-1]) == True:
    dfr = pd.concat([dfr, nowcomp])
    gdp_now = True
dfr['label'] = [f'Q{i.quarter} \\\\ {i.year}' if (i.quarter == 1) | (i == dfr.index[0]) 
                else dtxt(i)['qtr3'] for i in dfr.index]
dfr.to_csv(data_dir / 'gdp_comp_recent.csv', index_label='date')
dfrn = dfr.drop(['label', 'A191RL'], axis=1)

# Text for annual change latest year
ltyr = dfa.index[-1].year
totch = dfa['A191RL'].iloc[-1]
totcht = value_text(abs(totch), 'plain')
gdpt = (f'growth of {totcht} is' if totch > 0.1 
        else f'falling by {totcht} is' if totch < -0.1 
        else 'was unchanged as')
dfat = {name: value_text(val, 'plain', ptype='pp') 
        for name, val in dfa.iloc[-1].items()}
dfat['A019RY'] = value_text(dfa['A019RY'].iloc[-1], 
                            'plain', ptype='point') 
text = (f'In the latest full year of data, covering {ltyr}, GDP {gdpt} '+
        'the result of contributions from consumer '+
        f'spending of {dfat["DPCERY"]}, private investment of '+
        f'{dfat["A006RY"]}, government saving and investment of '+
        f'{dfat["A822RY"]}, and net exports of {dfat["A019RY"]}. ')
write_txt(text_dir / 'gdp_contrib_ltyr_exp.txt', text)
print(text, '\n')

# Text for chart on expenditure approach contributions
ltdt = dtxt(df.index[-1])['qtr1']
sl = [('DPCERY', 'contribution_to', False), ('A006RY', 'contribution', True), 
      ('A822RY', 'contribution', False), ('A019RY', 'contribution', True)]
d = {s: value_text(df[s].iloc[-1], style=style, ptype='pp', casual=cas,
                   digits=2, threshold=0.1) for s, style, cas in sl}
colors = {'DPCERY': 'yellow!80!orange', 'A006RY': 'blue!70!black', 
          'A822RY': 'cyan!50!white', 'A019RY': 'green!60!black'}
cl = {n: c_box(color) for n, color in colors.items()}
text = (f'In {ltdt}, consumer spending {cl["DPCERY"]} '+
        f'{d["DPCERY"]} real GDP growth. Private domestic '+
        f'investment {cl["A006RY"]} {d["A006RY"]}, '+
        f'government spending and investment {cl["A822RY"]} '+
        f'{d["A822RY"]}, and net exports {cl["A019RY"]} '+
        f'{d["A019RY"]}.')  
write_txt(text_dir / 'gdp_exp_basic.txt', text)
print(text)

# Value nodes
nodes = selected_nodes(dfrn, nowrow=gdp_now)
write_txt(text_dir / 'gdp_comp_recent_nodes.txt', nodes)
now_annote = ''
if gdp_now == True:
    now_annote = ('\draw [dashed] ({2023-10-01},\pgfkeysvalueof{/pgfplots/ymin}) -- '+
                  '({2023-10-01}, \pgfkeysvalueof{/pgfplots/ymax});\n'+
                  '\\node[anchor=west, align=left] at ({2023-10-01}, '+
                  '\pgfkeysvalueof{/pgfplots/ymax}) {\color{black!65}\\footnotesize '+
                  '{\\textit{GDPNow}}};\n')    
write_txt(text_dir / 'gdp_comp_recent_now_annote.txt', now_annote)

In the latest full year of data, covering 2022, GDP growth of 1.9 percent is the result of contributions from consumer spending of 1.7 percentage points, private investment of 0.9 percentage point, government saving and investment of negative 0.2 percentage point, and net exports of negative 0.5 point.  

In 2023 Q3, consumer spending (see\cbox{yellow!80!orange}) contributed 2.11 percentage points to real GDP growth. Private domestic investment (see\cbox{blue!70!black}) added 1.74 percentage points, government spending and investment (see\cbox{cyan!50!white}) contributed 0.99 percentage point, and net exports (see\cbox{green!60!black}) did not contribute.


### Labor Productivity ST / LT

In [17]:
# Sentence for comparison with median wage growth
df = fred_df('OPHNFB', start='1968')['VALUE']
lpch = ((df.iloc[-1] / df.loc['1989-01-01']) - 1) * 100
lpcht = value_text(lpch, 'increase_by')
text = ('Over this same period, which features a sharp increase '+
        f'in education in the US, labor productivity {lpcht}.')
write_txt(text_dir / 'lprod_rw_educ.txt', text)
print(text)

# Long-term and short-term growth rates
df = growth_rate(df)
lt = df.rolling(80).mean().dropna()
st = df.rolling(4).mean().dropna()
data = pd.DataFrame({'ST': st, 'LT': lt})
data.loc['1989':].to_csv(data_dir / 'prod_st_lt.csv', index_label='date')
ltdt = dtxt(data.index[-1])['qtr1']
ltavg = value_text(data['LT'].mean(), 'plain')
tty = value_text(data['LT'].iloc[-1], 'plain')
ltv = value_text(data['ST'].iloc[-1], 'plain')
colors = {'LT': 'orange!80!yellow', 'ST': 'blue!80!black'}
cl = {name: c_line(col) for name, col in colors.items()}
text = ('Over the longer-term, US labor productivity growth '+
        f'averages {ltavg} per year. The trailing 20-'+
        f'year average growth rate is {tty} in {ltdt} '+
        f'{cl["LT"]}. During the 1990s and early 2000s, labor '+
        'productivity growth was above its long-term average. '+
        'In contrast, from 2010 to 2017, productivity growth '+
        f'was below average. Over the year ending {ltdt}, '+
        f'growth averages {ltv} {cl["ST"]}. ')
write_txt(text_dir / 'prod_st_lt.txt', text)
print(text)

Over this same period, which features a sharp increase in education in the US, labor productivity increased by 94.4 percent.
Over the longer-term, US labor productivity growth averages two percent per year. The trailing 20-year average growth rate is 1.6 percent in 2023 Q3 (see {\color{orange!80!yellow}\textbf{---}}). During the 1990s and early 2000s, labor productivity growth was above its long-term average. In contrast, from 2010 to 2017, productivity growth was below average. Over the year ending 2023 Q3, growth averages 2.4 percent (see {\color{blue!80!black}\textbf{---}}). 


### Labor Productivity

In [18]:
# Series stored as a dictionary
series = {'PRS85006093': 'value_index',
          'PRS85006092': 'value',
          'PRS85006032': 'hours',
          'PRS85006042': 'output',
          'PRS85006033': 'hours_index',
          'PRS84006113': 'business_ulc',
          'PRS30006113': 'manuf_ulc',
          'PRS85006113': 'nfbus_ulc'}

In [19]:
# Start year and end year
#dates = (1989, 2023)
#df = bls_api(series, dates, bls_key)

#df.to_csv(data_dir / 'lprod.csv', index_label='date')

In [20]:
df = pd.read_csv(data_dir / 'lprod.csv', 
                 index_col='date', parse_dates=True)
ltdt = dtxt(df.index[-1])['qtr1']
prdt = dtxt(df.index[-2])['qtr1']
s = {srs: {'lt': value_text(df[srs].iloc[-1], adj='annual'), 
           'pr': value_text(df[srs].iloc[-2], adj='annual'), 
           'lt2': value_text(df[srs].iloc[-1], style='increase_of'), 
           'pr2': value_text(df[srs].iloc[-2]),
           'pr3': value_text(df[srs].iloc[-2], style='increase_of')} 
     for srs in series.values()}
grlt = cagr(df['value_index'])
grltt = value_text(grlt, 'plain')
gr5 = cagr(df['value_index'].iloc[-20:])
gr5t = value_text(gr5, adj='annual')
compare = compare_text(gr5, grlt, [0.1, 0.5, 2.0])

text = (f'In {ltdt}, nonfarm business labor productivity {s["value"]["lt"]} '+
        f'(see\cbox{{teal}}), as the result of {s["output"]["lt2"]} in '+
        f'real output and {s["hours"]["lt2"]} in hours worked. In the '+
        f'prior quarter, {prdt}, labor productivity {s["value"]["pr"]}, '+
        f'as real output {s["output"]["pr2"]} and hours of work '+
        f'{s["hours"]["pr2"]}. Productivity has {gr5t} over the past '+
        f'five years, {compare} the 1989-onward rate of {grltt}.')
write_txt(text_dir / 'lprod.txt', text)
print(text, '\n')

hr19 = value_text(cagr(df.loc['2017': '2019', 'hours_index']), adj='annual')
ch19txt = value_text(cagr(df.loc['2019-10-01':, 'hours_index']), 'increase_by')
prval = prval_comp(df['hours'])
text = (f'Total hours worked in nonfarm businesses {s["hours"]["lt"]} '+
        f'in {ltdt}, following {prval}. From '+
        f'2017 through 2019, total hours worked {hr19}. Since 2019, '+
        f'hours worked have {ch19txt} per year.')
write_txt(text_dir / 'tot_hours.txt', text)
print(text)

In 2023 Q3, nonfarm business labor productivity increased at an annual rate of 5.2 percent (see\cbox{teal}), as the result of an increase of 6.1 percent in real output and an increase of 0.9 percent in hours worked. In the prior quarter, 2023 Q2, labor productivity increased at an annual rate of 3.6 percent, as real output increased two percent and hours of work decreased 1.5 percent. Productivity has increased at an annual rate of 1.9 percent over the past five years, in line with the 1989-onward rate of 1.9 percent. 

Total hours worked in nonfarm businesses increased at an annual rate of 0.9 percent in 2023 Q3, following a decrease of 1.5 percent in Q2, and an increase of 2.6 percent in Q1. From 2017 through 2019, total hours worked increased at an annual rate of 1.3 percent. Since 2019, hours worked have increased by 0.7 percent per year.


### Financial Obligations Ratio

In [21]:
# Retrieve data and store
base = 'https://www.federalreserve.gov/datadownload/Output.aspx?'
srs = 'rel=FOR&series=1dc13603606b1a2cf3c07004eeb7f026&lastobs=&'
dt = 'from=01/01/1989&to=12/31/2023&'
oth = 'filetype=csv&label=include&layout=seriescolumn'
url = base + srs + dt + oth

d, clean_data = clean_fed_data(url)
d = {k: v.replace(', seasonally adjusted', '') for k, v in d.items()}
data = clean_data.rename(d, axis=1)
data.to_csv(data_dir / 'for.csv', index_label='date')

In [22]:
# End nodes and text
data = pd.read_csv(data_dir / 'for.csv', index_col='date', 
                   parse_dates=True)
# End nodes
cols = {'Financial obligations ratio': ['blue!80!black', 'q', 0.35], 
        'Debt service ratio': ['red', None, 0]}
nodes = '\n'.join([end_node(data[n], col, date=date, offset=offset) 
                   for n, [col, date, offset] in cols.items()])
write_txt(text_dir / 'for_nodes.txt', nodes)

# Text
fo = data['Financial obligations ratio']
ds = data['Debt service ratio']
ltdt = dtxt(data.index[-1])['qtr1']
ltval = fo.iloc[-1]
pryrval = fo.iloc[-5]
chval = value_text(ltval - pryrval, 'increase_of', ptype='pp', 
                   digits=2, threshold=0.03)
ch19 = value_text(ltval - fo.loc['2019-10-01'], 'increase_by', ptype='pp', 
                  digits=2, casual=True, threshold=0.03)
dsrval = ds.iloc[-1]
dch19 = value_text(dsrval - ds.loc['2019-10-01'], 'increase_by', ptype='pp', 
                   digits=2, casual=True, threshold=0.03).replace('was', 'is')
val90 = value_text(fo.loc['1990': '1999'].mean(), 'plain')
maxval = value_text(fo.max(), 'plain')
maxdt = dtxt(fo.idxmax())['qtr1']
hb = ', during the housing bubble' if fo.idxmax().year == 2007 else ''
m = data['Mortgage debt service ratio']
c = data['Consumer debt service ratio']
mltval = value_text(m.iloc[-1], 'plain')
cltval = value_text(c.iloc[-1], 'plain')
cl1 = c_line(cols['Financial obligations ratio'][0])
cl2 = c_line(cols['Debt service ratio'][0])

text = ('In the 1990s, financial obligations comprise an average of '+
        f'{val90} of income. This ratio peaked at {maxval} '+
        f'in {maxdt}{hb}.\n\n'+
        f'As of {ltdt}, household \\textbf{{financial obligations}} '+
        f'are {ltval:.1f} percent of income {cl1}, '+
        f'{chval} from the year prior. The financial obligations ratio '+
        f'{ch19} since 2019.\n\nIn the latest quarter, the ratio '+
        f'of \\textbf{{debt service}} payments to income is '+
        f'{dsrval:.1f} percent {cl2}. The debt service ratio '+
        f'{dch19} since 2019. In {ltdt}, the ratio of mortage debt '+
        f'service to income is {mltval}, and the ratio of '+
        f'consumer credit debt service to income is {cltval}.')
write_txt(text_dir / 'for.txt', text)
print(text)

In the 1990s, financial obligations comprise an average of 16.4 percent of income. This ratio peaked at 18.2 percent in 2007 Q4, during the housing bubble.

As of 2023 Q3, household \textbf{financial obligations} are 14.2 percent of income (see {\color{blue!80!black}\textbf{---}}), a decrease of 0.15 percentage point from the year prior. The financial obligations ratio fell by 0.52 percentage point since 2019.

In the latest quarter, the ratio of \textbf{debt service} payments to income is 9.8 percent (see {\color{red}\textbf{---}}). The debt service ratio fell by 0.20 percentage point since 2019. In 2023 Q3, the ratio of mortage debt service to income is four percent, and the ratio of consumer credit debt service to income is 5.8 percent.


### Durable goods new orders

In [23]:
# New orders for capital goods excluding defense or aircraft
url = ('https://api.census.gov/data/timeseries/eits/advm3?'+
       f'get=cell_value,time_slot_id&key={census_key}&'+
       'category_code=NXA&data_type_code=NO&time=from+1992&'+
       'for=us&seasonally_adj=yes')
r = requests.get(url).json()
date = lambda x: pd.to_datetime(x.time)
df = (pd.DataFrame(r[1:], columns=r[0]).assign(date = date)
        .set_index('date')['cell_value'].astype('float')).sort_index()
df.to_csv(data_dir / 'dgno_raw.csv', index_label='date', 
          header=True)

In [24]:
df = pd.read_csv(data_dir / 'dgno_raw.csv', index_col='date', 
                 parse_dates=True)['cell_value']
gdp = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])
res = ((df.resample('QS').sum() * 4  / 
        gdp['A191RC']).dropna() * 100).iloc[1:]
(res.rename('value').to_csv(data_dir / 'dgno.csv', 
                            index_label='date',  header=True))
color = 'purple!50!violet'
node = end_node(res, color, date='q')
write_txt(text_dir / 'dgno_node.txt', node)
ltval = f'\${df.iloc[-1] / 1000:,.0f} billion'
ldate = dtxt(df.index[-1])['mon1']
comp = pd.to_datetime('2020-02-01')
compdt = dtxt(pd.to_datetime(comp))['mon1']
val = value_text(df.pct_change(12).iloc[-1] * 100)
pcv = ((df.iloc[-1] / df.loc[comp]) - 1) * 100
pcval = value_text(pcv, 'increase_by')
text = ('New orders for manufactured core capital goods excluding '+
        f'aircraft total {ltval} in {ldate}, equivalent to '+
        f'{res.iloc[-1]:.1f} percent of GDP {c_line(color)}. New '+
        f'orders {val} over the past year, and {pcval} since '+
        f'{compdt}.')
write_txt(text_dir / 'dgno.txt', text)
print(text)

New orders for manufactured core capital goods excluding aircraft total \$74 billion in November 2023, equivalent to 3.2 percent of GDP (see {\color{purple!50!violet}\textbf{---}}). New orders increased two percent over the past year, and increased by 21.0 percent since February 2020.


### Shiller real return trailing 20-year average

In [25]:
url = 'http://www.econ.yale.edu/~shiller/data/ie_data.xls'
data = pd.read_excel(url, sheet_name='Data', header=7, 
                     index_col='Date').dropna(subset=['TR CAPE'])
data.index = pd.to_datetime(data.index.format())
data.loc['1989':, 'TR CAPE'].to_csv(data_dir / 'catrpe.csv', 
                                    index_label='date')
color = 'blue!80!cyan'
cl = c_line(color)
pe = data.loc['1989':, 'TR CAPE']
ltdt = dtxt(pe.index[-1])['mon1']
prdt = dtxt(pe.index[-2])['mon1']
yrdt = dtxt(pe.index[-13])['mon1']
ltval = pe.iloc[-1]
prval = pe.iloc[-2]
yrval = pe.iloc[-13]
val19 = pe.loc['2019'].mean()
val00 = pe.loc['2000'].mean()

text = (f'In {ltdt}, the Shiller total return CAPE ratio was '+
        f'{ltval:.1f}, compared to {prval:.1f} in {prdt} and '+
        f'{yrval:.1f} in {yrdt} {cl}. In 2019, the Shiller CAPE ratio '+
        f'was {val19:.1f}, on average. In 2000, during the stock market '+
        f'bubble, the ratio averaged {val00:.1f}. ')
write_txt(text_dir / 'cape.txt', text)
print(text)
node = end_node(data['TR CAPE'], color, date='m')
write_txt(text_dir / 'cape_node.txt', node)
col = ['Price', 'Dividend']
df = data.loc['1960':, col].dropna()
for yrs in [10, 15, 20]:
    mos = yrs * 12
    dy = (df.Dividend / df.Price).rolling(mos).mean()
    pch = (df.Price.pct_change(mos)+1)**(1/yrs) - 1
    df[f'r{yrs}'] = (dy + pch) * 100
    
res = df.loc['1989':, ~df.columns.isin(col)]
res.to_csv(data_dir / f'sp500rr.csv', index_label='date', 
           float_format='%g')
adj = node_adj(res)
smax = res.iloc[-1].idxmax()
adj[smax] = adj[smax] + 0.35

colors = {'r20': 'green!80!blue', 
          'r15': 'orange',
          'r10': 'blue'}
date = {series: 'm' if series == smax else None 
        for series in colors.keys()}
nodes  ='\n'.join([end_node(res[series], color, 
                            date=date[series], 
                            size=1.1, offset=adj[series]) 
                   for series, color in colors.items()])
write_txt(text_dir / 'sp500rr_nodes.txt', nodes)  
url = 'http://www.econ.yale.edu/~shiller/data.htm'
lt20 = df['r20'].iloc[-1]
ltdt = dtxt(df.index[-1])['mon1']
pr = df['r20'].loc['1995':'2005'].mean()
lt10 = df['r10'].iloc[-1]
pr10 = df['r10'].loc['1995':'2005'].mean()
cl = c_line(colors['r20'])
text = (f'According to historical stock market \href{{{url}}}'+
        '{data} from Robert Shiller, the \\textbf{inflation-adjusted '+
        'trailing twenty-year annual rate of return} of the S\&P '+
        f'500 is {lt20:.1f} percent as of {ltdt} {cl}. Ultra-long-'+
        'term real returns are currently low relative to the '+
        'average trailing twenty-year real annual return of '+
        f'{pr:.1f} percent during 1995--2005. The trailing ten-'+
        f'year real return was {lt10:.1f} percent, as of {ltdt}, '+
        f'and {pr10:.1f} percent during 1995--2005 '+
        f'{c_line(colors["r10"])}. ')
write_txt(text_dir / 'sp500rr.txt', text)
print(text)

In September 2023, the Shiller total return CAPE ratio was 33.3, compared to 32.9 in August 2023 and 30.6 in September 2022 (see {\color{blue!80!cyan}\textbf{---}}). In 2019, the Shiller CAPE ratio was 32.1, on average. In 2000, during the stock market bubble, the ratio averaged 45.1. 
Three conflicting nodes
According to historical stock market \href{http://www.econ.yale.edu/~shiller/data.htm}{data} from Robert Shiller, the \textbf{inflation-adjusted trailing twenty-year annual rate of return} of the S\&P 500 is 6.9 percent as of June 2023 (see {\color{green!80!blue}\textbf{---}}). Ultra-long-term real returns are currently low relative to the average trailing twenty-year real annual return of 10.1 percent during 1995--2005. The trailing ten-year real return was 9.3 percent, as of June 2023, and 10.7 percent during 1995--2005 (see {\color{blue}\textbf{---}}). 


In [26]:
dy = ((data.D / data.P).dropna() * 100).loc['1989':].rename('DY')
dy.to_csv(data_dir / 'sp500dy.csv', index_label='date')
color = 'green!80!black'

node = end_node(dy, color, date='m', digits=2, offset=0.3)
write_txt(text_dir / 'sp500dy_node.txt', node)

ltdt = dtxt(dy.index[-1])['mon1']
prdt = dtxt(dy.index[-2])['mon1']
yrdt = dtxt(dy.index[-13])['mon1']
ltval = dy.iloc[-1]
prval = dy.iloc[-2]
yrval = dy.iloc[-13]
cl = c_line(color)
avg = dy.loc['1990': '2015'].mean()
text = (f'In {ltdt}, the dividend yield for the S\&P 500 is '+
        f'{ltval:.2f} percent {cl}, compared to {prval:.2f} percent '+
        f'in {prdt}, and {yrval:.2f} percent in {yrdt}. From 1990 to '+
        f'2015, the dividend yield averaged {avg:.2f} percent.')
write_txt(text_dir / 'sp500div.txt', text)
print(text)

In June 2023, the dividend yield for the S\&P 500 is 1.58 percent (see {\color{green!80!black}\textbf{---}}), compared to 1.65 percent in May 2023, and 1.64 percent in June 2022. From 1990 to 2015, the dividend yield averaged 2.09 percent.


### High Quality Corporate Bond Yield

In [27]:
df = fred_df('HQMCB10YR')

# Spread between Corporate Bonds and Treasuries 
dft = pd.read_csv(data_dir / 'treas_raw.csv', index_col='date', 
                 parse_dates=True).loc['1989':, 'Ten-year']

df['Treasury'] = dft.resample('MS').mean()
df['Spread'] = df['VALUE'] - df['Treasury']
df[['VALUE', 'Spread']].to_csv(data_dir / 'hqcb.csv', 
                               index_label='date')

color = 'violet!80!black'
node = end_node(df['VALUE'], color, date='m', digits=2)
write_txt(text_dir / 'hqcb_node.txt', node)

data = df.VALUE.apply('{:.2f} percent'.format).values
idx = [dtxt(i)['mon1'] for i in df.index]
cl = c_line(color)

text = (f'The yield on high-quality corporate bonds with a maturity '+
        f'of 10 years is {data[-1]} in {idx[-1]}, following '+
        f'{data[-2]} in {idx[-2]} {cl}. One year prior, in '+
        f'{idx[-13]}, this spot rate was {data[-13]}, and '+
        f'four years prior, in {idx[-49]}, it was {data[-49]}.')
write_txt(text_dir / 'hqcb.txt', text)
print(text)

The yield on high-quality corporate bonds with a maturity of 10 years is 5.10 percent in December 2023, following 5.72 percent in November 2023 (see {\color{violet!80!black}\textbf{---}}). One year prior, in December 2022, this spot rate was 5.17 percent, and four years prior, in December 2019, it was 2.92 percent.


### International Investment Position (IIP)

In [28]:
# Annual GDP for 1988-2005
s = 'A191RC'
r = bea_api_nipa([f'T10105'], bea_key, freq='A')
data = json.loads(r[0][2])['BEAAPI']['Results']
date = lambda x: (pd.to_datetime(x.TimePeriod) + 
                  pd.DateOffset(months=6))
value = lambda x: x.DataValue.str.replace(',','')
gdpa = (pd.DataFrame(data['Data']).query('SeriesCode == @s')
          .assign(date = date, value=value).set_index('date')
          .loc[:'2005', 'value'].astype(int))
gdpq = nipa_df(retrieve_table('T10105')['Data'], [s])[s]
gdp = pd.concat([gdpa, gdpq.loc['2006':]])

srs = ['FinAssets', 'FinLiabs', 'Net']
years = ','.join(map(str, range(1988, 2024)))
res = pd.DataFrame()
for s in srs:
    url = (f'https://apps.bea.gov/api/data/?&UserID={bea_key}'+
           f'&method=GetData&datasetname=IIP&TypeOfInvestment={s}'+
           f'&Component=Pos&Frequency=A,QNSA&Year={years}')
    r = requests.get(url)
    t = pd.DataFrame(r.json()['BEAAPI']['Data'])
    a = t.query('Frequency == "A"')[['TimePeriod', 'DataValue']]
    a = a.set_index(pd.to_datetime(a['TimePeriod']) + 
                    pd.DateOffset(months=6)).loc[:'2005', 'DataValue']
    q = t.query('Frequency == "QNSA"')[['TimePeriod', 'DataValue']]
    q = q.set_index(pd.to_datetime(q['TimePeriod']))['DataValue']
    res[s] = pd.concat([a, q]).astype('float')

sh = res.divide(gdp, axis=0).multiply(100).dropna()
sh.loc['1989':].to_csv(data_dir / 'iip.csv', index_label='date')

tot = res.iloc[-1] / 1_000_000
lt = sh.iloc[-1]
ltdt = dtxt(sh.index[-1])['qtr1']
pr = sh.iloc[-2]
prdt = dtxt(sh.index[-2])['qtr1']
pr19 = sh.loc['2019'].mean()
col_a, col_l, col_n = 'blue!95!violet', 'red', 'cyan!25!white'
cb = c_box(col_n).replace('see ', 'see')
text = (f'In {ltdt}, domestic holdings of foreign assets total '+
        f'\${tot.FinAssets:.1f} trillion, which is {lt.FinAssets:.1f} '+
        f'percent of GDP {c_line(col_a)}. These assets translate to '+
        f'{pr.FinAssets:.1f} percent of GDP in {prdt}, and '+
        f'{pr19.FinAssets:.1f} percent in 2019. '+
        'Domestic liabilities to the foreign sector total '+
        f'\${tot.FinLiabs:.1f} trillion, or {lt.FinLiabs:.1f} percent of '+
        f'GDP, in {ltdt}, following {pr.FinLiabs:.1f} percent in {prdt}, '+
        f'and {pr19.FinLiabs:.1f} percent in 2019 {c_line(col_l)}.\n\n'+
        f'The overall result of these financial positions, net IIP, or '+
        'holdings of foreign assets minus liabilities, identifies the '+
        'US as a net debtor to the rest of the world, to the equivalent '+
        f'of {abs(lt.Net):.1f} percent of GDP in {ltdt}, following '
        f'{abs(pr.Net):.1f} percent in {prdt}, and {abs(pr19.Net):.1f} '+
        f'percent in 2019 {cb}.')
write_txt(text_dir / 'niip.txt', text)
print(text)

In 2023 Q3, domestic holdings of foreign assets total \$32.9 trillion, which is 119.2 percent of GDP (see {\color{blue!95!violet}\textbf{---}}). These assets translate to 123.8 percent of GDP in 2023 Q2, and 128.8 percent in 2019. Domestic liabilities to the foreign sector total \$51.1 trillion, or 185.0 percent of GDP, in 2023 Q3, following 190.9 percent in 2023 Q2, and 180.2 percent in 2019 (see {\color{red}\textbf{---}}).

The overall result of these financial positions, net IIP, or holdings of foreign assets minus liabilities, identifies the US as a net debtor to the rest of the world, to the equivalent of 65.8 percent of GDP in 2023 Q3, following 67.0 percent in 2023 Q2, and 51.4 percent in 2019 (see\cbox{cyan!25!white}).


### H.6 Money Stock - M2 (Monthly)

In [29]:
url = ('https://www.federalreserve.gov/datadownload/Output.aspx?'+
       'rel=H6&series=411c4c269dc600450339f8d4809d80eb&lastobs=&'+
       'from=01/01/1987&to=12/31/2023&filetype=csv&label=include&'+
       'layout=seriescolumn')
d, df = clean_fed_data(url)
df.rename(d, axis=1).to_csv(data_dir / 'h6raw.csv', index_label='date')

In [30]:
df = pd.read_csv(data_dir / 'h6raw.csv', index_col='date', 
                 parse_dates=True)
ltdate = dtxt(df.index[-1])['mon1']
prmodt = dtxt(df.index[-2])['mon1']
ltval = df['M2'].iloc[-1] / 1000.0
one_yr = value_text(df['M2'].pct_change(12).iloc[-1] * 100, threshold=0.1)
pr_mo = value_text(df['M2'].pct_change(12).iloc[-2] * 100, 
                   style='increase_of')
four_yr = value_text(df['M2'].pct_change(48).iloc[-1] * 100)

# M2 share of GDP
m2q = df.M2.resample('QS').mean() * 1_000
gdp = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])['A191RC']
m2gdp = ((m2q / gdp) * 100).dropna()
m2gi = (m2gdp / m2gdp.iloc[0]).rename('value')
m2gi.to_csv(data_dir / 'm2gdp.csv', index_label='date',
         float_format='%g')
ltqtr = dtxt(m2gi.index[-1])['qtr2']
lt = m2gdp.iloc[-1]
totch = (m2gi.iloc[-1] - 1) * 100

txt1 = (f'In {ltdate}, the M2 money stock totals \${ltval:.1f} '+
        'trillion. Put into the context of overall economic activity, '+
        f'M2 is equivalent to {lt:.1f} percent of GDP in {ltqtr}.\n\nDuring '+
        'the 1990s, the ratio of money to economic activity was falling '+
        f'{c_line("magenta")}. Following the great recession, the money '+
        'supply has expanded relative to activity. Since 1989, the ratio '+
        f'has increased by a total of {totch:.1f} percent.')
write_txt(text_dir / 'm2lvl.txt', txt1)
col = 'green!80!blue'

mo_ch = value_text((df.M2.pct_change()*100).iloc[-1], threshold=0.1)
prval = prval_comp(df.M2.pct_change()*100)
txt2 = (f'The M2 money stock {mo_ch} in {ltdate}, over the previous '+
        f'month, following {prval}. Over the past 12 '+
        f'months, the money stock {one_yr} {c_line(col)}. The M2 money '+
        f'stock has {four_yr}, in total, over the past four years. ')
write_txt(text_dir / 'm2chg.txt', txt2)
print(txt1, '\n', txt2)

r = pd.DataFrame({'value': df['M2'].pct_change(12) * 100,
                  '3M': m3rate(df['M2']),
                  '1M': (((df.M2.pct_change() + 1) ** 12) - 1) * 100}
                ).loc['1989':]
r.value.to_csv(data_dir / 'm2.csv', index_label='date', header=True,
         float_format='%g')

node = end_node(r.value, col, offset=0.35, date='m')
write_txt(text_dir / 'm2_node.txt', node)

In November 2023, the M2 money stock totals \$20.8 trillion. Put into the context of overall economic activity, M2 is equivalent to 75.1 percent of GDP in the third quarter of 2023.

During the 1990s, the ratio of money to economic activity was falling (see {\color{magenta}\textbf{---}}). Following the great recession, the money supply has expanded relative to activity. Since 1989, the ratio has increased by a total of 32.8 percent. 
 The M2 money stock increased 0.4 percent in November 2023, over the previous month, following decreases of 0.1 percent in both October and September. Over the past 12 months, the money stock decreased three percent (see {\color{green!80!blue}\textbf{---}}). The M2 money stock has increased 36.0 percent, in total, over the past four years. 


### Consumer Credit (G.19)

The same chart also includes a line from the Z.1 notebook

In [31]:
base = 'https://www.federalreserve.gov/datadownload/Output.aspx?'
srs = 'rel=G19&series=8c7511a37e4aea678be81e7a61df57db&lastobs=&'
dt = 'from=01/01/1989&to=12/31/2023&'
oth = 'filetype=csv&label=include&layout=seriescolumn'
url = base + srs + dt + oth

d, df = clean_fed_data(url)
cc = df['DTCTL.M']

dpi = (pd.read_csv(data_dir / 'nipa20600.csv', index_col='date', 
                   parse_dates=True).loc['1989':, 'A067RC'])
rate = ((cc / dpi) * 100).rename('value').dropna()
rate.to_csv(data_dir / 'cc_dpi_monthly.csv', index_label='date')

node_color = 'blue!50!black'
node = end_node(rate, node_color, date='m', offset=0.35)
write_txt(text_dir / 'cc_dpi_monthly_node.txt', node)

ltdate = dtxt(cc.index[-1])['mon1']
ltval = cc.iloc[-1] / 1_000_000
ltrate = rate.iloc[-1]
one_yr = value_text(cc.pct_change(12).iloc[-1] * 100, style='increase_by')
one_yr_dpi = value_text(dpi.pct_change(12).iloc[-1] * 100, style='increase_by')
one_yr_rate = value_text(rate.diff(12).iloc[-1], adj='total', 
                         ptype='pp', threshold=0.1)
cline = c_line(node_color)
also = 'also ' if one_yr == one_yr_dpi else ''    
text = ('In the monthly measure, consumer credit totals '+
        f'\${ltval:.2f} trillion US dollars on a '+
        f'seasonally-adjusted and annualized basis in {ltdate}. '+
        f'Over the past year, consumer credit {one_yr}, while '+
        f'after-tax income {also}{one_yr_dpi}. As a result, the ratio '+
        f'of consumer credit to disposable income {one_yr_rate}. '+
        f'In {ltdate}, total consumer credit is equivalent to '+
        f'{ltrate:.1f} percent of annualized {ltdate} disposable '+
        f'income {cline}. ')
write_txt(text_dir / 'cc_dpi.txt', text)
print(text)

In the monthly measure, consumer credit totals \$5.01 trillion US dollars on a seasonally-adjusted and annualized basis in November 2023. Over the past year, consumer credit increased by 2.8 percent, while after-tax income increased by seven percent. As a result, the ratio of consumer credit to disposable income decreased by a total of one percentage point. In November 2023, total consumer credit is equivalent to 24.4 percent of annualized November 2023 disposable income (see {\color{blue!50!black}\textbf{---}}). 


### Employment Cost Index

In [32]:
series = {'CIU2020000000000A': 'WS',
          'CIU2030000000000A': 'Be',
          'CIS1020000000000I': 'IndexWS',
          'CIS102G000000000I': 'IndexWSGoods',
          'CIU2020000000000I': 'IndexWS_PVT_NSA',
          'CIU1020000000000I': 'IndexWS_ALL_NSA',
          'ECU20001I': 'IndexWS_ALL_NSA_SIC',
          'ECS20001I': 'IndexWS_All_SA_SIC'}

#dates = (1988, 2023)
#df = bls_api(series, dates, bls_key)
#df.to_csv(data_dir/ 'eci.csv', index_label='date')

In [33]:
df = pd.read_csv(data_dir / 'eci.csv', index_col='date', 
                 parse_dates=True)[['WS', 'Be']]

adj = node_adj(df)
smax = df.iloc[-1].idxmax()
adj[smax] = adj[smax] + 0.35

colors = {'WS': 'green!75!blue!60!black', 
          'Be': 'cyan!90!blue'}
date = {series: 'q' if series == smax else None 
        for series in colors.keys()}
nodes  ='\n'.join([end_node(df[series], color, 
                            date=date[series], 
                            size=1.1, offset=adj[series])
                   for series, color in colors.items()])
write_txt(text_dir / 'eci_nodes.txt', nodes)

obs = [(-1, 'lt'), (-2, 'pr'), (-3, 'pr2')]
sty = [('increase', '1'), ('increase_of', '2'), 
       ('increase', '3')]
dt = {n: dtxt(df.index[i]) for i, n in obs}
v = {f'{name}_{n}_{s}': value_text(val, style=style, threshold=0.1) 
     for i, n in obs for name, val 
     in df.iloc[i].to_dict().items() 
     for style, s in sty}

valpr = prval_comp(df.WS)
val19 = {s: value_text(df.loc['2019', s].mean(), 'increase_by') 
         for s in ['WS', 'Be']}
text = (f'Over the year ending {dt["lt"]["qtr1"]}, private '+
        f'industry wage and salary costs {v["WS_lt_3"]} '+
        f'{c_line(colors["WS"])}, '+
        f'following {valpr}. In '+
        f'2019, private wages and salaries {val19["WS"]}.'+
        '\n\nThe cost of private sector benefits '+
        f'{v["Be_lt_3"]} {c_line(colors["Be"])} over the year '+
        f'ending {dt["lt"]["qtr1"]}, following {v["Be_pr_2"]} '+
        f'in {dt["pr"]["qtr1"]}. In 2019, private-sector '+
        f'benefits costs {val19["Be"]}.')
write_txt(text_dir / 'eci.txt', text)
print(text, '\n')

# Calculate Gross Labor Income
start = '2017-01-01'
eci = pd.read_csv(data_dir / 'eci.csv', index_col='date', 
                  parse_dates=True)['WS']
epop = (pd.read_csv(data_dir / 'jobs_report_main.csv', index_col='date', 
                   parse_dates=True)['PA_EPOP'].pct_change(12)
        .resample('QS').mean()) * 100
ece = (eci + epop).rename('ECI_EPOP').dropna()
# Shift dates to center quarter on chart
ds = ece.loc[start:]
ds.index = ds.index + pd.DateOffset(days=45)
ds.to_csv(data_dir / 'gli_qtr.csv', index_label='date')

#text
cline = c_line('violet!80!black')
ltdt = dtxt(ece.index[-1])['qtr1']
text = ('Calculating gross labor income from the employment cost index '+
        f'for private industries and the prime age employment rate {cline}, '+
        f'one-year growth is {ece.iloc[-1]:.1f} percent in {ltdt}, '+
        f'following {ece.iloc[-5]:.1f} percent one year prior. ')
write_txt(text_dir / 'gli_eci.txt', text)
print(text)

Over the year ending 2023 Q3, private industry wage and salary costs increased 4.5 percent (see {\color{green!75!blue!60!black}\textbf{---}}), following increases of 4.6 percent in Q2 and 5.1 percent in Q1. In 2019, private wages and salaries increased by three percent.

The cost of private sector benefits increased 3.9 percent (see {\color{cyan!90!blue}\textbf{---}}) over the year ending 2023 Q3, following an increase of 3.9 percent in 2023 Q2. In 2019, private-sector benefits costs increased by two percent. 

Calculating gross labor income from the employment cost index for private industries and the prime age employment rate (see {\color{violet!80!black}\textbf{---}}), one-year growth is 5.4 percent in 2023 Q3, following 8.0 percent one year prior. 


In [34]:
# Latest data for wage summary chart
df = pd.read_csv(data_dir / 'eci.csv', index_col='date', 
                 parse_dates=True)
cpi = pd.read_csv(data_dir / 'cpi_raw.csv', 
                 index_col='date', parse_dates=True)
cpidt = cpi.index[-73]
df.loc[cpidt:, 'WS'].to_csv(data_dir / 'eci_growth_recent.csv', 
                      index_label='date')

# Long-term chart
df = pd.read_csv(data_dir / 'eci.csv', index_col='date', 
                 parse_dates=True)
res = (df[['IndexWS_ALL_NSA_SIC', 'IndexWS_ALL_NSA']]
       .pct_change(4, fill_method=None).multiply(100))

res.loc['1989':].to_csv(data_dir / 'eci_yy.csv', 
                      index_label='date')

node = end_node(res['IndexWS_ALL_NSA'], 'blue!80!purple!80!white', 
                date='q', offset=0.35)
write_txt(text_dir / 'eci_yy_node.txt', node)

In [35]:
# Quarterly growth chart
df = pd.read_csv(data_dir / 'eci.csv', index_col='date', 
                 parse_dates=True)[['IndexWS']]
res = (df.pct_change() * 400)

# deflator
d = nipa_df(retrieve_table('T20304')['Data'], ['DPCERG'])['DPCERG']
pr = (d.pct_change() * 400)

# Calculate real series
res = pd.concat([res, pr.to_frame()], axis=1)
res['REAL'] = res['IndexWS'] - res['DPCERG']
res['Quarter'] = [f'Q{i.quarter}\\\\{i.year}' if i.quarter == 1 
                  else dtxt(i)['qtr3'] for i in res.index]

# Save results
res.iloc[-13:].to_csv(data_dir / 'eci_qq.csv', index_label='date')

# End node for real series
node = end_node(res.REAL, 'violet', size=1.8, xoffset=0.06)
write_txt(text_dir / 'eci_qq_real_node.txt', node)

# Text
ltdt = dtxt(res.index[-1])['qtr1']

cb = c_box('cyan!50!lime!70!white')
cl = c_line('violet')

nomlt = value_text(res.IndexWS.iloc[-1])
valpr = prval_comp(res.IndexWS)
rlt = value_text(res.REAL.iloc[-1], 'plain')
rvalpr = prval_comp(res.REAL)

ct1 = compare_text(res.IndexWS.iloc[-1], 
             res.loc['2021', 'IndexWS'].max(), 
             [0.2, 0.6, 1.2], plain=True)

ct2 = compare_text(res.IndexWS.iloc[-1], 
             res.loc['2019', 'IndexWS'].mean(), 
             [0.2, 0.6, 1.2], plain=True)

ct3 = compare_text(res.REAL.iloc[-1], 
             res['REAL'].mean(), 
             [0.2, 0.6, 1.2], plain=True)

text = (f'In {prdt}, wages {nomlt}, following '+
        f'{valpr} {cb}. '+
        f'Growth is currently {ct1} the 2021 highs and {ct2} '+
        'the pre-COVID average.\n\nAdjusted for inflation using '+
        f'the PCE deflator, growth is {rlt} in {ltdt}, '+
        f'following {rvalpr} {cl}. Real wage growth is {ct3} '+
        'the long-term average. ')
write_txt(text_dir / 'eci_qq.txt', text)
print(text)

In 2023 Q2, wages increased 4.8 percent, following increases of 3.8 percent in Q2 and 4.6 percent in Q1 (see\cbox{cyan!50!lime!70!white}). Growth is currently below the 2021 highs and above the pre-COVID average.

Adjusted for inflation using the PCE deflator, growth is 2.2 percent in 2023 Q3, following increases of 1.4 percent in Q2 and 0.5 percent in Q1 (see {\color{violet}\textbf{---}}). Real wage growth is above the long-term average. 


### Treasury International Capital

In [36]:
# URL 
file = ('https://treasury.gov/resource-center/data-chart-center/tic/'+
        'Documents/npr_history.csv')
#file = ('/home/brian/Downloads/npr_history.csv')
# Match names with numbers of columns in report
names = {10: 'Treasuries_official', 5: 'Treasuries_private', 
         11: 'Agencies_official', 6: 'Agencies_private', 
         12: 'Corporate_official', 7: 'Corporate_private',
         13: 'Equities_official', 8: 'Equities_private'}

df = pd.read_csv(file, header=12, index_col=0, parse_dates=True).iloc[1:].dropna()
df.index.name = 'date'
df.index = pd.to_datetime(df.index)
rn = {f'[{k}]': v for k, v in names.items()}
res = df[rn.keys()].rename(rn, axis=1).sort_index().loc['1988':]
# Categories to combine
c = {'Treasury_Bonds': ['Treasuries_official',
                        'Treasuries_private'],
     'Agency_Bonds': ['Agencies_official', 
                      'Agencies_private'],
     'Corporate_Bonds': ['Corporate_official',
                         'Corporate_private'],
     'Equities': ['Equities_official', 'Equities_private']}

final = pd.DataFrame()
for name, grp in c.items():
    final[name] = (res[grp].astype('int').sum(axis=1).sort_index())

In [37]:
s = ['Treasury_Bonds', 'Agency_Bonds', 'Corporate_Bonds']
pce = pd.read_csv(data_dir / 'nipa20804.csv', index_col='date', 
                  parse_dates=True).loc[final.index, 'DPCERG']
pr = pce / pce.iloc[-1]
data = (final[s].rolling(12).sum().loc['1989':]
        .divide(1000).dropna())
adj = (final[s].divide(pr, axis=0).rolling(12).sum().loc['1989':]
        .divide(1000).dropna())
adj.to_csv(data_dir/ 'tic_bond.csv', index_label='date')
date = dtxt(adj.index[-1])['mon1']
write_txt(text_dir / 'tic_date.txt', date)
print(date)

October 2023


In [38]:
ltdt = dtxt(data.index[-1])['mon1']
gdp = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])['A191RC']
tbgdp = ((data.iloc[-1].loc['Treasury_Bonds'] * 1_000) / 
         gdp.iloc[-1]) * 100
tbgdpt = value_text(tbgdp, 'plain')

colors = {'Treasury_Bonds': 'black!25!white', 
          'Corporate_Bonds': 'blue!70!white',
          'Agency_Bonds': 'orange!30!yellow!90!white'}

cb = {k: c_box(v).replace('see ', 'see') for k,v in colors.items()}

d = {}
for i in s:
    item = i.replace('_', ' ').lower()
    val = data.iloc[-1].loc[i]
    buysell = 'buyer' if val > 0 else 'seller'
    txt = (f'a net {buysell} of \${val:,.0f} billion of US {item}' 
           if abs(val) >= 5 else 
           f'about unchanged in holdings of US {item}')
    d[i] = txt

text = (f'Over the year ending {ltdt}, the rest of the world was '+
        f'{d["Treasury_Bonds"]}, equivalent to {tbgdpt} '+
        f'of US GDP {cb["Treasury_Bonds"]}. Over the same period, '+
        f'the rest of the world was {d["Agency_Bonds"]}, '+
        f'{cb["Agency_Bonds"]}, and {d["Corporate_Bonds"]}, '+
        f'{cb["Corporate_Bonds"]}.')
write_txt(text_dir / 'tic.txt', text)
print(text)

Over the year ending October 2023, the rest of the world was a net buyer of \$502 billion of US treasury bonds, equivalent to 1.8 percent of US GDP (see\cbox{black!25!white}). Over the same period, the rest of the world was a net buyer of \$172 billion of US agency bonds, (see\cbox{orange!30!yellow!90!white}), and a net buyer of \$213 billion of US corporate bonds, (see\cbox{blue!70!white}).


### FRBNY Survey of Consumer Expectations Job Separation Expectation

In [39]:
url = ('https://www.newyorkfed.org/medialibrary/interactives/'+
       'sce/sce/downloads/data/frbny-sce-data.xlsx')
df = pd.read_excel(url, sheet_name='Job separation expectation', 
                   skiprows=3, index_col=0)
df.index = pd.to_datetime([f'{str(i)[:4]}-{str(i)[-2:]}-01' 
                          for i in df.index])
df.columns = ['Losing', 'Leaving']
df.to_csv(data_dir / 'sce_job_separation.csv', index_label='date')

In [40]:
df = pd.read_csv(data_dir / 'sce_job_separation.csv', index_col='date', 
                 parse_dates=True)

lcol = 'cyan'
vcol = 'violet!50!blue'
vnode = end_node(df.Leaving, vcol, date='m', offset=0.35)
lnode = end_node(df.Losing, lcol)
nodes = '\n'.join([vnode, lnode])
write_txt(text_dir / 'sce_job_sep_nodes.txt', nodes)

ltdt = dtxt(df.index[-1])['mon1']
ltv = df.Leaving.iloc[-1]
prv = df.Leaving.loc['2019'].mean()
ltl = df.Losing.iloc[-1]
prl = df.Losing.loc['2019'].mean()
diff = ltv - ltl
diffpr = prv - prl
exfb = 'exceed' if diff > 0 else 'false below'
text = (f'In {ltdt}, the perceived likelihood of leaving '+
        "one's job voluntarily in the next 12 months "+
        f'averages {ltv:.1f} percent, compared to {prv:.1f} '+
        f'percent in 2019 {c_line(vcol)}. In the latest month, '+
        f"the perceived probability losing one's job is {ltl:.1f} "+
        f'percent, compared to {prl:.1f} percent in 2019 '+
        f'{c_line(lcol)}.\n\nDuring the pandemic, in April 2020, '+
        'job loss expectations exceeded job leaving expectations. '+
        f'In {ltdt}, job leaving expectations {exfb} job loss '+
        f'expectations by {diff:.1f} percentage points, compared '+
        f'to {diffpr:.1f} percentage points in 2019. ')
write_txt(text_dir / 'sce_job_sep.txt', text)
print(text)

In December 2023, the perceived likelihood of leaving one's job voluntarily in the next 12 months averages 20.4 percent, compared to 21.0 percent in 2019 (see {\color{violet!50!blue}\textbf{---}}). In the latest month, the perceived probability losing one's job is 13.4 percent, compared to 14.3 percent in 2019 (see {\color{cyan}\textbf{---}}).

During the pandemic, in April 2020, job loss expectations exceeded job leaving expectations. In December 2023, job leaving expectations exceed job loss expectations by 7.0 percentage points, compared to 6.8 percentage points in 2019. 


### Real Effective Exchange Rate

From BIS

Index 2010 = 100

In [41]:
df = fred_df('RBUSBIS')
df.to_csv(data_dir / 'reer.csv', index_label='date')
ltdt = dtxt(df.index[-1])['mon1']
ltval = value_text(df.VALUE.iloc[-1] - 100)
val19 = df.loc['2019', 'VALUE'].mean()
lt3m = df.VALUE.iloc[-3:].mean()
url = 'https://www.bis.org/statistics/eer.htm'
text = (f'The Bank for International Settlements (BIS) \href{{{url}}}{{calculates}} '+
        '\\textbf{real effective exchange rates} for many countries, on a monthly '+
        f'basis. As of {ltdt}, the US dollar real effective exchange rate has '+
        f'{ltval} since 2010. In 2019, the index average was {val19:.1f}. Over the '+
        f'past three months, the index average value was {lt3m:.1f}.')
write_txt(text_dir / 'reer.txt', text)
print(text)
node = end_node(df.VALUE, 'red', date='m', full_year=True, offset=-0.35)
write_txt(text_dir / 'reer_node.txt', node)

The Bank for International Settlements (BIS) \href{https://www.bis.org/statistics/eer.htm}{calculates} \textbf{real effective exchange rates} for many countries, on a monthly basis. As of December 2023, the US dollar real effective exchange rate has increased 6.6 percent since 2010. In 2019, the index average was 98.6. Over the past three months, the index average value was 108.5.


### Construction Spending

Census data on the value of construction put-in-place

In [42]:
# Category codes and data type codes
d = {'PrRes': ['A00XX', 'V'], 'PrNR': ['ANRXX', 'V'], 
     'Pub': ['AXXXX', 'P'], 'Total': ['AXXXX', 'T'],
     'PrMan': ['A20IX', 'V']}
date = lambda x: pd.to_datetime(x.time)
df = pd.DataFrame()
for name, (cc, dtc) in d.items():
    url = ('https://api.census.gov/data/timeseries/eits/vip?'+
           f'get=cell_value,time_slot_id&key={census_key}&time=from+2002&'+
           f'category_code={cc}&data_type_code={dtc}&'+
           'for=us&seasonally_adj=yes')
    r = requests.get(url).json()
    df[name] = (pd.DataFrame(r[1:], columns=r[0]).assign(date = date)
                .set_index('date')['cell_value'].astype('float')).sort_index()
df.to_csv(data_dir / 'construction_spending.csv', index_label='date')

In [43]:
df = pd.read_csv(data_dir / 'construction_spending.csv', index_col='date', 
                 parse_dates=True)

gdp = pd.read_csv(data_dir / 'gdp_monthly.csv', index_col='date', 
                  parse_dates=True)['A191RC']

sh = (df.divide(gdp, axis=0).dropna() * 100)
sh.to_csv(data_dir / 'construction_spending_sh.csv', index_label='date')

# Text for total
ltdt = dtxt(sh.index[-1])['mon1']
ltval = f'\${(df.Total.iloc[-1] / 1_000_000):.1f} trillion'
ltsh = value_text(sh.Total.iloc[-1], 'eq', adj='GDP')

text = (f'In {ltdt}, the annualized value of construction put-in-place '+
        f'is {ltval}, {ltsh}.')
write_txt(text_dir / 'construction_spending_total.txt', text)
print(text)

# End nodes
colors = {'PrRes': 'blue!60!white', 'PrNR': 'yellow!60!orange', 
          'Pub': 'violet!60!black'}
res = sh[colors.keys()]
adj = node_adj(res)
smax = res.iloc[-1].idxmax()
adj[smax] = adj[smax] + 0.35

date = {series: 'm' if series == smax else None 
        for series in colors.keys()}
nodes  ='\n'.join([end_node(res[series], color, 
                            date=date[series], full_year=True, 
                            size=1.1, offset=adj[series]) 
                   for series, color in colors.items()])
write_txt(text_dir / 'construction_spending_nodes.txt', nodes)  

# Share of GDP by sector
shd = {i: value_text(sh[i].iloc[-1], 'plain') 
       for i in ['PrRes', 'PrNR', 'Pub']}
cl = {i: c_line(c) for i, c in colors.items()}

# Growth contribution
df['GDP'] = gdp
gc = growth_contrib_ann(df, 'GDP', freq='M')
gcd = {i: value_text(gc[i].iloc[-1], 'contribution', 
                     ptype='pp', digits=2) 
       for i in ['PrRes', 'PrNR', 'Pub', 'Total']}
gcnr = value_text(gc['PrNR'].iloc[-1], 'contribution', 
                   ptype=None, digits=2) + ' point'
gcpub = value_text(gc['Pub'].iloc[-1], 'contribution', casual=True,
                   ptype=None, digits=2) + ' point'

text = (f'By sector, private residential construction is '+
        f'{shd["PrRes"]} of GDP {cl["PrRes"]} in {ltdt}, '+
        f'private nonresidential construction is {shd["PrNR"]} '+
        f'{cl["PrNR"]}, and government construction is '+
        f'{shd["Pub"]} {cl["Pub"]}.\n\n'+
        'Over the past year, construction spending '+
        f'{gcd["Total"]} to nominal GDP growth. Private residential '+
        f'construction {gcd["PrRes"]}, private '+
        f'nonresidential {gcnr}, and '+
        f'public construction {gcpub}.')
write_txt(text_dir / 'construction_spending_sector.txt', text)  
print(text)

In November 2023, the annualized value of construction put-in-place is \$2.1 trillion, equivalent to 7.4 percent of GDP.
By sector, private residential construction is 3.2 percent of GDP (see {\color{blue!60!white}\textbf{---}}) in November 2023, private nonresidential construction is 2.5 percent (see {\color{yellow!60!orange}\textbf{---}}), and government construction is 1.6 percent (see {\color{violet!60!black}\textbf{---}}).

Over the past year, construction spending contributed 0.79 percentage point to nominal GDP growth. Private residential construction contributed 0.12 percentage point, private nonresidential contributed 0.43 point, and public construction added 0.24 point.
